# Calculation of Marginal Likelihood

## Likelihood from NLE

In [2]:
# Essentials
from scipy.special import logsumexp
import numpy as np
import pandas as pd
import pickle
import sbi.utils as utils
import torch

lik = pickle.load(open(f'posteriors/posterior_chuong_snle_10.pkl', 'rb')) # Amortized Likelihood estimator
posterior_chuong = pickle.load(open(f'posteriors/posterior_chuong.pkl', 'rb')) # Amortized posterior estimator

# Model Prior
prior_min = np.log10(np.array([1e-2,1e-7,1e-8]))
prior_max = np.log10(np.array([1,1e-2,1e-2]))
prior = utils.BoxUniform(low=torch.tensor(prior_min), 
                         high=torch.tensor(prior_max))

## $P(X) = \int P(X|\zeta)P(\zeta)d\zeta$

In [9]:
# Function for P(X)
def get_PX(lik, prior, x, n):
    # Explored space
    s = np.linspace(prior.base_dist.low[0], prior.base_dist.high[0], n)
    m = np.linspace(prior.base_dist.low[1], prior.base_dist.high[1], n)
    p = np.linspace(prior.base_dist.low[2], prior.base_dist.high[2], n)

    # Create empty grid
    grd = torch.tensor([[[[s_,m_,p_,0] for s_ in s] for m_ in m] for p_ in p], dtype=torch.float32).reshape(n**3,4)
    # Fill it with likelihood (potential = likelihood)
    grd[:,3] = lik.potential(x=x,theta=grd[:,0:3]) # vectorized
    # lens = np.array([float(prior.base_dist.high[i])-float(prior.base_dist.low[i]) for i in range(len(prior.base_dist.high))]) # Prior dimensions
    # A = np.prod(lens) # Prior volume -> P(θ) = 1/A
    # dt = A / (n**3) # Granularity
    # y = grd[:,3] + np.log(1/A) + np.log(dt)
    return float(logsumexp(grd[:,3]))# Riemann sum ~ integral -> marginal likelihood

lines = ['wt','ltr','ars','all']
cont_df = pd.DataFrame(index = lines, columns = [f'rep {i+1}' for i in range(8)])
for l in range(len(lines)):
    line = lines[l]
    X = pd.read_csv(f'empirical_data/{line}.csv', index_col=0) # unimputed data
    conts = [round(get_PX(lik, prior, X.iloc[i,:], 100)) for i in range(len(X))]
    cont_df.iloc[l,:len(conts)] = conts
cont_df.replace(np.nan, '') # aesthetics

/home/jupyter-nadavbennun/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks.py:575: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = atleast_2d(torch.as_tensor(x, dtype=float32))
/home/jupyter-nadavbennun/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks.py:575: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = atleast_2d(torch.as_tensor(x, dtype=float32))
/home/jupyter-nadavbennun/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks.py:575: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as label

,rep 1,rep 2,rep 3,rep 4,rep 5,rep 6,rep 7,rep 8
wt,-1950,-4174,-1080,-22527,-1544,,,
ltr,-2466,-1497,-12121,-940,-257,-139,-287,
ars,-5349,-3610,-3446,-352,-1760,-4671,-4475,
all,-898,-1051,-5150,-1839,-3161,-2111,-893,-1635


wt #4 and ltr #3 are possibly unrepresentative

## $P(X) = { {P(X|\theta)P(\theta)} \over {P(\theta|X)} } \forall \theta$

In [5]:
from sbi.inference import MCMCPosterior

def est_PX(posterior, lik, x, n_samples):
    if x.name in ['gap1_ars_1', 'gap1_ars_3']:
        potential_fn = posterior.potential_fn
        potential_fn.set_x(x)
        posterior_mcmc = MCMCPosterior(potential_fn, proposal = prior)
        samples = posterior_mcmc.set_default_x(x).sample((n_samples,))
       
    else:
        samples = posterior.set_default_x(x).sample((n_samples,))
        
    post_vals = posterior.set_default_x(x).log_prob(samples)
    lik_vals = lik.set_default_x(x).potential(samples)
    return float((logsumexp(lik_vals-post_vals)-np.log(n_samples)).round())

lines = ['wt','ltr','ars','all']
cont_df = pd.DataFrame(index = lines, columns = [f'rep {i+1}' for i in range(8)])
maps = pd.read_csv('maps/sbi_all_maps_chuong.csv', index_col=0)

for l in range(len(lines)):
    line = lines[l]
    X = pd.read_csv(f'empirical_data/{line}.csv', index_col=0) # unimputed data
    conts = [est_PX(posterior_chuong, lik, X.iloc[i,:], 100) for i in range(len(X))]
    cont_df.iloc[l,:len(conts)] = conts
cont_df.replace(np.nan, '') # aesthetics

/home/jupyter-nadavbennun/.local/lib/python3.9/site-packages/sbi/utils/user_input_checks.py:575: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  x = atleast_2d(torch.as_tensor(x, dtype=float32))


Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

...

Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

Drawing 100 posterior samples:   0%|          | 0/100 [00:00<?, ?it/s]

/tmp/ipykernel_725567/1308329093.py:26: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cont_df.replace(np.nan, '') # aesthetics


,rep 1,rep 2,rep 3,rep 4,rep 5,rep 6,rep 7,rep 8
wt,-2319.0,-9939.0,-1167.0,-26800.0,-2586.0,,,
ltr,-6712.0,-2522.0,-inf,-1504.0,-594.0,-169.0,-1774.0,
ars,-133143.0,-33901.0,-25178.0,-422.0,-6917.0,-57112.0,-29552.0,
all,-1698.0,-2582.0,-49282.0,-5065.0,-9784.0,-7617.0,-1507.0,-6394.0
